# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM, PAE_ELM
from nitools.models import LeNet5, ResNet18
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [2]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

### pAE-RVFL

In [20]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_rvfl = PAE_RVFL(
    784, 
    400, 
    mnist_class, 
    subnets=1,
    device=device, 
    r=(1,1), sc=0.5, sb=0.5, c=10, ae_iters=3)

lenet = LeNet5(pae_rvfl, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

train: 21.289515018463135
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.961 P=0.961 R=0.961 F1=0.961
              precision    recall  f1-score   support

         0.0      0.991     0.962     0.976      1009
         1.0      0.989     0.959     0.974      1171
         2.0      0.952     0.967     0.959      1015
         3.0      0.963     0.958     0.961      1016
         4.0      0.951     0.974     0.962       959
         5.0      0.955     0.961     0.958       887
         6.0      0.961     0.965     0.963       954
         7.0      0.942     0.969     0.955       999
         8.0      0.954     0.951     0.952       977
         9.0      0.948     0.945     0.947      1013

    accuracy                          0.961     10000
   macro avg      0.961     0.961     0.961     10000
weighted avg      0.961     0.961     0.961     10000


Confusion matrix:
 [[ 971    0    1    0    0    1    3    1    3    0]
 [   0 1123    2    2    0  

### ELM

In [9]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = AE_ELM.AE_ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=100,
    output_size=mnist_class,
    ae_iters=3)

lenet = ResNet18.ResNet18(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:5000]
y = torch.from_numpy(mnist['train_y']).float()[:5000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet AE-ELM - MNIST', pred_arg, mnist['test_y'])

RuntimeError: The size of tensor a (10) must match the size of tensor b (6) at non-singleton dimension 3

In [ ]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = ELM.ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=1000,
    output_size=mnist_class)

lenet = LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet  - MNIST', pred_arg, mnist['test_y'])

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [ ]:
# Import dataset
cifar10 = utils.load_cifar10()

cifar_in = 32*32*3
cifar_class = 10

### SPAE-RVFL

In [ ]:
device = torch.device("cpu")

pae_rvfl = SPAE_RVFL.SPAE_RVFL(
    cifar_in, 
    cifar_class, 
    [100], 
    subnets=[5], 
    device=device, 
    r=[(1,1), (1,1)], sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5_3D(pae_rvfl)

X = torch.from_numpy(cifar10['train_X']).float()[:5000]
y = torch.from_numpy(cifar10['train_y']).float()[:5000]

result = lenet.train(X, y)
tX = torch.from_numpy(cifar10['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, cifar10['test_y'])

In [ ]:
# Show results

In [24]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_elm = PAE_ELM(
    784, 
    700, 
    mnist_class, 
    subnets=10,
    device=device, 
    c=10, ae_iters=5)

lenet = LeNet5(pae_elm, weight_train=False)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-ELM - MNIST', pred_arg, mnist['test_y'])

train: 139.8301920890808
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.981 P=0.981 R=0.980 F1=0.980
              precision    recall  f1-score   support

         0.0      0.991     0.982     0.986       989
         1.0      0.992     0.991     0.992      1136
         2.0      0.978     0.983     0.981      1026
         3.0      0.978     0.984     0.981      1004
         4.0      0.976     0.988     0.982       970
         5.0      0.982     0.978     0.980       896
         6.0      0.986     0.982     0.984       962
         7.0      0.976     0.974     0.975      1030
         8.0      0.980     0.970     0.975       985
         9.0      0.966     0.973     0.970      1002

    accuracy                          0.981     10000
   macro avg      0.981     0.980     0.980     10000
weighted avg      0.981     0.981     0.981     10000


Confusion matrix:
 [[ 971    1    1    0    0    2    3    1    1    0]
 [   0 1126    1    2    1   